# #MakeoverMonday - Music Industry Sales
> This visualisation examines how digital changed the music industry.

- toc: false 
- badges: true
- comments: true
- categories: [makeovermonday, altair, python]
- image: images/music_industry.png

## 40 Years of Music Industry Sales

The record industry has seen a lot of change over the years.

8-tracks took a short-lived run at the dominance of vinyl, cassettes faded away as compact discs took the world by storm, and through it all, the music industry saw its revenue continue to climb. That is, until it was digitally disrupted.

Looking back at four decades of U.S. music industry sales data is a fascinating exercise as it charts not only the rise and fall the record company profits, but seismic shifts in technology and consumer behavior as well.

Sources: https://www.visualcapitalist.com/music-industry-sales/, https://www.riaa.com/u-s-sales-database/<br>
Full script: https://github.com/lb930/Data-Visualisation/tree/master/MakeoverMonday/MM%20W21-2020%20Music%20Industry%20Sales

In [42]:
# hide
import altair as alt
import pandas as pd
from altair_saver import save

In [43]:
# hide
df = pd.read_csv("MusicData.csv", parse_dates=["Year"], thousands=',')
df

﻿Format            Metric       Year     Value  Category
0               CD             Units 1973-01-01       NaN  Physical
1               CD             Units 1974-01-01       NaN  Physical
2               CD             Units 1975-01-01       NaN  Physical
3               CD             Units 1976-01-01       NaN  Physical
4               CD             Units 1977-01-01       NaN  Physical
...            ...               ...        ...       ...       ...
2088  Vinyl Single  Value (Adjusted) 2015-01-01  6.205390  Physical
2089  Vinyl Single  Value (Adjusted) 2016-01-01  5.198931  Physical
2090  Vinyl Single  Value (Adjusted) 2017-01-01  6.339678  Physical
2091  Vinyl Single  Value (Adjusted) 2018-01-01  5.386197  Physical
2092  Vinyl Single  Value (Adjusted) 2019-01-01  6.795946  Physical

[2093 rows x 5 columns]

In [44]:
# hide
# Convert Value column to float so it can be used in Altair as quantitative measure
df.astype({"Value": "float"})
df["Value"].dtype

dtype('float64')

In [45]:
# hide
# Check for leading or trailing whitespace in columns
df.columns

Index(['﻿Format', 'Metric', 'Year', 'Value', 'Category'], dtype='object')

In [46]:
# hide
# Get an overview of the format which have been used
df.groupby(['﻿Format']).sum()

Value
﻿Format                                          
8 - Track                            23640.469715
CD                                  305524.451086
CD Single                             2446.015682
Cassette                            102312.105824
Cassette Single                       4177.126130
DVD Audio                               84.535783
Download Album                       12658.615697
Download Music Video                   316.854561
Download Single                      17199.061065
Kiosk                                   54.802373
LP/EP                                94233.655807
Limited Tier Paid Subscription        2487.659115
Music Video (Physical)               10619.480336
On-Demand Streaming (Ad-Supported)    4159.447624
Other Ad-Supported Streaming           866.632070
Other Digital                           77.586600
Other Tapes                            230.180841
Paid Subscription                    21731.722104
Ringtones & Ringbacks                 6216.661824
SACD                                   101.752622
SoundExchange Distributions           7413.514033
Synchronization                       2566.364051
Vinyl Single                         14629.378953

In [47]:
# hide
# Create a new column which assigns groups to each format. These groups will be displayed in the stream chart


def groups(row):
    if row["﻿Format"] == "8 - Track":
        val = "8 - Track"
    elif (
        row["﻿Format"] == "Cassette"
        or row["﻿Format"] == "Cassette Single"
        or row["﻿Format"] == "Other Tapes"
    ):
        val = "Tape"
    elif (
        row["﻿Format"] == "CD"
        or row["﻿Format"] == "CD Single"
        or row["﻿Format"] == "DVD Audio"
    ):
        val = "CD"
    elif (
        row["﻿Format"] == "Download Album"
        or row["﻿Format"] == "Download Music Video"
        or row["﻿Format"] == "Download Single"
        or row["﻿Format"] == "Kiosk"
    ):
        val = "Download"
    elif (
        row["﻿Format"] == "Music Video (Physical)"
        or row["﻿Format"] == "Other Digital"
        or row["﻿Format"] == "Music Video (Physical)"
        or row["﻿Format"] == "Ringtones & Ringbacks"
        or row["﻿Format"] == "SACD"
        or row["﻿Format"] == "SoundExchange Distributions"
        or row["﻿Format"] == "Synchronization"
    ):
        val = "Other"
    elif (
        row["﻿Format"] == "Limited Tier Paid Subscription"
        or row["﻿Format"] == "On-Demand Streaming (Ad-Supported)"
        or row["﻿Format"] == "Other Ad-Supported Streaming"
        or row["﻿Format"] == "Paid Subscription"
    ):
        val = "Stream"
    else:
        val = "Vinyl"
    return val

In [48]:
# hide
df["Format group"] = df.apply(groups, axis=1)
df

﻿Format            Metric       Year     Value  Category  \
0               CD             Units 1973-01-01       NaN  Physical   
1               CD             Units 1974-01-01       NaN  Physical   
2               CD             Units 1975-01-01       NaN  Physical   
3               CD             Units 1976-01-01       NaN  Physical   
4               CD             Units 1977-01-01       NaN  Physical   
...            ...               ...        ...       ...       ...   
2088  Vinyl Single  Value (Adjusted) 2015-01-01  6.205390  Physical   
2089  Vinyl Single  Value (Adjusted) 2016-01-01  5.198931  Physical   
2090  Vinyl Single  Value (Adjusted) 2017-01-01  6.339678  Physical   
2091  Vinyl Single  Value (Adjusted) 2018-01-01  5.386197  Physical   
2092  Vinyl Single  Value (Adjusted) 2019-01-01  6.795946  Physical   

     Format group  
0              CD  
1              CD  
2              CD  
3              CD  
4              CD  
...           ...  
2088        Vinyl  
2089        Vinyl  
2090        Vinyl  
2091        Vinyl  
2092        Vinyl  

[2093 rows x 6 columns]

In [49]:
# hide
# Check if all rows have been assigned groups
df.groupby(['Format group']).sum()

Value
Format group               
8 - Track      23640.469715
CD            308055.002552
Download       30229.333696
Other          26995.359465
Stream         29245.460913
Tape          106719.412794
Vinyl         108863.034759

In [50]:
selection = alt.selection_multi(fields=["Format group"], bind="legend")

chart = alt.Chart(df).mark_area().configure_axis(grid=False).encode(
    alt.X("yearmonth(Year):T", title="", axis=alt.Axis(format="%Y", domain=False, tickSize=0)),
    alt.Y("sum(Value):Q", stack="center", axis=None),
    alt.Color(
        "Format group:N",
        scale=alt.Scale(
            domain=[
                "8 - Track",
                "CD",
                "Vinyl",
                "Tape",
                "Other",
                "Download",
                "Stream",
            ],
            range=["#423e58", "#d3587b", "#6a3589", "#9b478b", "#ffae78", "#f78972", "#fbcd98"],
        ),
    ),
    tooltip=["Format group", "sum(Value):Q", "year(Year):T"],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
).add_selection(selection)

chart

alt.Chart(...)

In [51]:
# hide
chart.save("music_industry.html")